<a href="https://colab.research.google.com/github/shrenikraxit/T-100-Analysis/blob/main/Hypothesis_3_The_Great_Hub_Hierarchy_Inversion_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hypothesis_3_The_Great_Hub_Hierarchy_Weakening

In [ ]:
import pandas as pd
import numpy as np

# Step 0 - Environment preparation

# Connect with google drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
#!/usr/bin/env python3
"""
Hypothesis 3: Hub Hierarchy Inversion Analysis
Testing: Traditional airport hierarchy demonstrates systematic inversion with smaller airports outperforming larger ones

This script performs comprehensive statistical analysis to examine whether the traditional
airport hierarchy is inverting, with smaller airports demonstrating superior growth performance
compared to larger airports in the post-COVID recovery period.

Statistical Analysis Framework:
- Inverse correlation analysis between airport size and growth performance
- Hierarchical performance matrix verification
- Market concentration analysis using Herfindahl-Hirschman Index
- Statistical significance testing for hierarchy inversion patterns

Author: Aviation Recovery Research
Date: 2025
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import warnings
warnings.filterwarnings('ignore')

def main():
    print("=== HYPOTHESIS 3: HUB HIERARCHY INVERSION ANALYSIS ===")
    print("Testing: Traditional airport hierarchy demonstrates systematic inversion with smaller airports outperforming larger ones")
    print()

    # Data Loading from Previous Analysis
    print("Data Loading and Initial Processing...")

    try:
        # Load the airport analysis from Hypothesis 1
        airport_data = pd.read_csv('/content/drive/MyDrive/airline_data_analysis_v2/hyp1_outputs/hypothesis1_updated_airport_analysis.csv')
        print(f"Loaded airport analysis data: {len(airport_data):,} airports")

        # Load growth by hub results
        hub_growth = pd.read_csv('/content/drive/MyDrive/airline_data_analysis_v2/hyp1_outputs/hypothesis1_updated_growth_by_hub.csv', index_col=0)
        print(f"Loaded hub growth data: {len(hub_growth)} hub types")

    except FileNotFoundError:
        print("Error: Hypothesis 1 results not found. Please run Hypothesis 1 analysis first.")
        return None

    # Prepare data for hierarchy analysis
    meaningful_airports = airport_data[
        (airport_data['PASSENGERS_PRE'] >= 10000) &
        (airport_data['PASSENGERS_POST'] >= 10000) &
        (airport_data['HUB_TYPE'].notna())
    ].copy()

    print(f"Airports included in hierarchy analysis: {len(meaningful_airports):,}")
    print(f"Hub types represented: {meaningful_airports['HUB_TYPE'].unique()}")
    print()

    # Metric 1: Inverse Correlation Analysis
    print("METRIC 1: INVERSE CORRELATION ANALYSIS")
    print("=" * 60)

    def perform_correlation_analysis():
        """Analyze correlation between airport size and growth performance"""

        # Remove outliers for more robust correlation analysis
        growth_data = meaningful_airports['CHANGE_PERCENT']
        size_data = meaningful_airports['PASSENGERS_PRE']

        # Calculate percentiles for outlier removal
        growth_q1, growth_q3 = growth_data.quantile([0.25, 0.75])
        growth_iqr = growth_q3 - growth_q1
        growth_lower = growth_q1 - 1.5 * growth_iqr
        growth_upper = growth_q3 + 1.5 * growth_iqr

        size_q1, size_q3 = size_data.quantile([0.25, 0.75])
        size_iqr = size_q3 - size_q1
        size_lower = size_q1 - 1.5 * size_iqr
        size_upper = size_q3 + 1.5 * size_iqr

        # Filter outliers
        outlier_mask = (
            (growth_data >= growth_lower) & (growth_data <= growth_upper) &
            (size_data >= size_lower) & (size_data <= size_upper)
        )

        filtered_airports = meaningful_airports[outlier_mask].copy()

        print(f"Original sample size: {len(meaningful_airports)}")
        print(f"Sample size after outlier removal: {len(filtered_airports)}")
        print(f"Outliers removed: {len(meaningful_airports) - len(filtered_airports)} ({((len(meaningful_airports) - len(filtered_airports)) / len(meaningful_airports) * 100):.1f}%)")
        print()

        # Perform correlation analyses
        if len(filtered_airports) >= 10:
            # Pearson correlation (linear relationship)
            pearson_corr, pearson_p = pearsonr(filtered_airports['PASSENGERS_PRE'],
                                              filtered_airports['CHANGE_PERCENT'])

            # Spearman correlation (monotonic relationship)
            spearman_corr, spearman_p = spearmanr(filtered_airports['PASSENGERS_PRE'],
                                                 filtered_airports['CHANGE_PERCENT'])

            print("CORRELATION ANALYSIS RESULTS:")
            print("-" * 40)
            print(f"Pearson correlation coefficient: {pearson_corr:.4f}")
            print(f"Pearson p-value: {pearson_p:.6f}")
            print(f"Spearman correlation coefficient: {spearman_corr:.4f}")
            print(f"Spearman p-value: {spearman_p:.6f}")
            print()

            # Interpret correlation strength and direction
            def interpret_correlation(corr_coef):
                if abs(corr_coef) < 0.1:
                    return "negligible"
                elif abs(corr_coef) < 0.3:
                    return "weak"
                elif abs(corr_coef) < 0.5:
                    return "moderate"
                else:
                    return "strong"

            pearson_strength = interpret_correlation(pearson_corr)
            spearman_strength = interpret_correlation(spearman_corr)

            print("CORRELATION INTERPRETATION:")
            print(f"Pearson correlation strength: {pearson_strength}")
            print(f"Spearman correlation strength: {spearman_strength}")

            if pearson_corr < 0 and pearson_p < 0.05:
                print("Statistical result: Significant negative correlation - smaller airports demonstrate superior growth")
            elif pearson_corr > 0 and pearson_p < 0.05:
                print("Statistical result: Significant positive correlation - larger airports demonstrate superior growth")
            else:
                print("Statistical result: No significant correlation between airport size and growth")

            return {
                'pearson_corr': pearson_corr,
                'pearson_p': pearson_p,
                'spearman_corr': spearman_corr,
                'spearman_p': spearman_p,
                'sample_size': len(filtered_airports),
                'outliers_removed': len(meaningful_airports) - len(filtered_airports)
            }

        else:
            print("Warning: Insufficient sample size for correlation analysis")
            return None

    correlation_results = perform_correlation_analysis()
    print()

    # Metric 2: Hierarchical Performance Matrix
    print("METRIC 2: HIERARCHICAL PERFORMANCE MATRIX")
    print("=" * 60)

    def create_performance_matrix():
        """Create comprehensive performance matrix by hub type"""

        # Calculate hub-level statistics
        hub_stats = meaningful_airports.groupby('HUB_TYPE').agg({
            'PASSENGERS_PRE': ['count', 'mean', 'sum'],
            'PASSENGERS_POST': 'sum',
            'CHANGE_PERCENT': ['mean', 'median', 'std']
        }).round(2)

        # Flatten column names
        hub_stats.columns = ['Airport_Count', 'Avg_Size_Pre', 'Total_Pre', 'Total_Post',
                            'Avg_Growth', 'Median_Growth', 'Std_Growth']

        # Calculate aggregate growth rates
        hub_stats['Aggregate_Growth_Rate'] = ((hub_stats['Total_Post'] - hub_stats['Total_Pre']) /
                                             hub_stats['Total_Pre'] * 100).round(2)

        # Calculate winner statistics
        winner_stats = meaningful_airports.groupby('HUB_TYPE').apply(
            lambda x: pd.Series({
                'Winners': (x['CHANGE_PERCENT'] >= 5).sum(),
                'Stable': ((x['CHANGE_PERCENT'] > -5) & (x['CHANGE_PERCENT'] < 5)).sum(),
                'Losers': (x['CHANGE_PERCENT'] <= -5).sum()
            })
        )

        # Calculate winner rates
        winner_stats['Total_Airports'] = winner_stats.sum(axis=1)
        winner_stats['Winner_Rate'] = (winner_stats['Winners'] / winner_stats['Total_Airports'] * 100).round(1)

        # Combine results
        performance_matrix = pd.merge(hub_stats, winner_stats[['Winner_Rate']], left_index=True, right_index=True)

        # Define theoretical hierarchy (by size, largest to smallest)
        size_hierarchy = ['Major Hub', 'Medium Hub', 'Small Hub', 'Non-Hub Primary', 'Regional/Local']
        performance_hierarchy = performance_matrix.sort_values('Avg_Growth', ascending=False).index.tolist()

        print("HIERARCHICAL PERFORMANCE MATRIX:")
        print("-" * 80)
        print(f"{'Hub Type':<15} {'Count':<6} {'Avg Size (K)':<12} {'Avg Growth':<11} {'Winner %':<9} {'Aggregate %':<11}")
        print("-" * 80)

        for hub_type in performance_hierarchy:
            if hub_type in performance_matrix.index:
                row = performance_matrix.loc[hub_type]
                avg_size = row['Avg_Size_Pre'] / 1000  # Convert to thousands
                print(f"{hub_type:<15} {int(row['Airport_Count']):<6} {avg_size:<12,.0f} "
                      f"{row['Avg_Growth']:<11.1f} {row['Winner_Rate']:<9.1f} "
                      f"{row['Aggregate_Growth_Rate']:<11.1f}")

        print()
        print("HIERARCHY COMPARISON:")
        print("-" * 40)
        print("Traditional hierarchy (by size):")
        for i, hub_type in enumerate(size_hierarchy, 1):
            if hub_type in performance_matrix.index:
                print(f"  {i}. {hub_type}")

        print("\nPerformance hierarchy (by growth):")
        for i, hub_type in enumerate(performance_hierarchy, 1):
            print(f"  {i}. {hub_type}")

        # Check for inversion
        hierarchy_inverted = False
        if len(performance_hierarchy) >= 3:
            # Check if smaller hubs are performing better than larger hubs
            small_hub_rank = performance_hierarchy.index('Small Hub') if 'Small Hub' in performance_hierarchy else 999
            major_hub_rank = performance_hierarchy.index('Major Hub') if 'Major Hub' in performance_hierarchy else 999

            if small_hub_rank < major_hub_rank:  # Lower rank = better performance
                hierarchy_inverted = True
                print(f"\nHIERARCHY INVERSION DETECTED: Small hubs (rank {small_hub_rank + 1}) outperform major hubs (rank {major_hub_rank + 1})")
            else:
                print(f"\nNO HIERARCHY INVERSION: Traditional hierarchy maintained")

        return performance_matrix, hierarchy_inverted

    performance_matrix, hierarchy_inverted = create_performance_matrix()
    print()

    # Metric 3: Market Concentration Analysis
    print("METRIC 3: MARKET CONCENTRATION ANALYSIS")
    print("=" * 60)

    def calculate_market_concentration():
        """Calculate Herfindahl-Hirschman Index for market concentration"""

        def calculate_hhi(passenger_shares):
            """Calculate HHI from passenger shares (as percentages)"""
            return sum(share**2 for share in passenger_shares)

        # Calculate market shares for pre-COVID and post-COVID
        total_pre = meaningful_airports['PASSENGERS_PRE'].sum()
        total_post = meaningful_airports['PASSENGERS_POST'].sum()

        pre_shares = (meaningful_airports['PASSENGERS_PRE'] / total_pre * 100).values
        post_shares = (meaningful_airports['PASSENGERS_POST'] / total_post * 100).values

        hhi_pre = calculate_hhi(pre_shares)
        hhi_post = calculate_hhi(post_shares)
        hhi_change = hhi_post - hhi_pre

        print("MARKET CONCENTRATION ANALYSIS (Herfindahl-Hirschman Index):")
        print("-" * 65)
        print(f"Pre-COVID HHI: {hhi_pre:.1f}")
        print(f"Post-COVID HHI: {hhi_post:.1f}")
        print(f"HHI change: {hhi_change:+.1f}")
        print()

        # Interpret HHI values
        def interpret_hhi(hhi_value):
            if hhi_value < 1500:
                return "unconcentrated"
            elif hhi_value < 2500:
                return "moderately concentrated"
            else:
                return "highly concentrated"

        pre_concentration = interpret_hhi(hhi_pre)
        post_concentration = interpret_hhi(hhi_post)

        print("CONCENTRATION INTERPRETATION:")
        print(f"Pre-COVID market: {pre_concentration}")
        print(f"Post-COVID market: {post_concentration}")

        if hhi_change < -50:
            print("Market concentration change: Significant deconcentration")
        elif hhi_change < -10:
            print("Market concentration change: Moderate deconcentration")
        elif hhi_change < 0:
            print("Market concentration change: Slight deconcentration")
        elif hhi_change > 50:
            print("Market concentration change: Significant concentration increase")
        elif hhi_change > 10:
            print("Market concentration change: Moderate concentration increase")
        else:
            print("Market concentration change: Minimal change")

        # Calculate concentration by hub type
        hub_concentration = meaningful_airports.groupby('HUB_TYPE').agg({
            'PASSENGERS_PRE': 'sum',
            'PASSENGERS_POST': 'sum'
        })

        hub_concentration['Market_Share_Pre'] = (hub_concentration['PASSENGERS_PRE'] / total_pre * 100).round(2)
        hub_concentration['Market_Share_Post'] = (hub_concentration['PASSENGERS_POST'] / total_post * 100).round(2)
        hub_concentration['Share_Change'] = (hub_concentration['Market_Share_Post'] -
                                           hub_concentration['Market_Share_Pre']).round(2)

        print()
        print("MARKET SHARE BY HUB TYPE:")
        print("-" * 50)
        print(f"{'Hub Type':<15} {'Pre-COVID %':<11} {'Post-COVID %':<12} {'Change':<8}")
        print("-" * 50)

        for hub_type, row in hub_concentration.iterrows():
            change_indicator = "+" if row['Share_Change'] >= 0 else ""
            print(f"{hub_type:<15} {row['Market_Share_Pre']:<11.2f} {row['Market_Share_Post']:<12.2f} "
                  f"{change_indicator}{row['Share_Change']:<8.2f}")

        return {
            'hhi_pre': hhi_pre,
            'hhi_post': hhi_post,
            'hhi_change': hhi_change,
            'hub_concentration': hub_concentration
        }

    concentration_results = calculate_market_concentration()
    print()

    # Hypothesis Evaluation
    print("HYPOTHESIS EVALUATION")
    print("=" * 60)

    print("HYPOTHESIS: Traditional airport hierarchy demonstrates systematic inversion with smaller airports outperforming larger ones")
    print()

    # Success criteria assessment
    criteria_met = 0

    # Criterion 1: Negative correlation between size and growth
    print("CRITERION 1: Negative Size-Growth Correlation")
    if correlation_results:
        pearson_corr = correlation_results['pearson_corr']
        pearson_p = correlation_results['pearson_p']

        print(f"  Pearson correlation: {pearson_corr:.4f} (p = {pearson_p:.6f})")

        if pearson_corr < 0 and pearson_p < 0.05:
            print("  CRITERION 1 MET: Significant negative correlation between airport size and growth")
            criteria_met += 1
        elif pearson_corr < 0:
            print("  CRITERION 1 PARTIALLY MET: Negative correlation but not statistically significant")
        else:
            print("  CRITERION 1 NOT MET: No negative correlation between size and growth")
    else:
        print("  CRITERION 1 CANNOT BE EVALUATED: Correlation analysis failed")

    print()

    # Criterion 2: Hierarchy inversion in performance rankings
    print("CRITERION 2: Hierarchy Performance Inversion")

    # Calculate growth advantage from existing data
    if 'Small Hub' in performance_matrix.index and 'Major Hub' in performance_matrix.index:
        small_hub_growth = performance_matrix.loc['Small Hub', 'Avg_Growth']
        major_hub_growth = performance_matrix.loc['Major Hub', 'Avg_Growth']
        growth_advantage = small_hub_growth - major_hub_growth

        print(f"  Small Hub average growth: {small_hub_growth:.1f}%")
        print(f"  Major Hub average growth: {major_hub_growth:.1f}%")
        print(f"  Small hub advantage: {growth_advantage:+.1f} percentage points")

        if hierarchy_inverted and growth_advantage > 2:
            print("  CRITERION 2 MET: Clear hierarchy inversion with meaningful difference")
            criteria_met += 1
        elif hierarchy_inverted:
            print("  CRITERION 2 PARTIALLY MET: Hierarchy inversion but small difference")
        else:
            print("  CRITERION 2 NOT MET: No hierarchy inversion")
    else:
        print("  CRITERION 2 CANNOT BE EVALUATED: Insufficient hub type data")

    print()

    # Criterion 3: Market deconcentration
    print("CRITERION 3: Decreased Market Concentration")
    hhi_change = concentration_results['hhi_change']

    print(f"  HHI change: {hhi_change:+.1f}")

    if hhi_change < -50:
        print("  CRITERION 3 STRONGLY MET: Significant deconcentration (HHI decrease > 50)")
        criteria_met += 1
    elif hhi_change < -10:
        print("  CRITERION 3 MET: Meaningful deconcentration (HHI decrease > 10)")
        criteria_met += 1
    elif hhi_change < 0:
        print("  CRITERION 3 PARTIALLY MET: Some deconcentration but minimal")
    else:
        print("  CRITERION 3 NOT MET: No deconcentration")

    print()

    # Final Verdict
    print("FINAL VERDICT:")
    print("=" * 40)
    print(f"Criteria met: {criteria_met}/3")

    if criteria_met == 3:
        print("HYPOTHESIS STRONGLY SUPPORTED: Strong evidence for hub hierarchy inversion")
        print("The traditional airport hierarchy demonstrates clear systematic inversion")
    elif criteria_met == 2:
        print("HYPOTHESIS SUPPORTED: Good evidence for hub hierarchy inversion")
        print("Significant evidence that smaller airports outperform larger ones")
    elif criteria_met == 1:
        print("HYPOTHESIS PARTIALLY SUPPORTED: Limited evidence for hierarchy inversion")
        print("Some indicators support the hypothesis but evidence is mixed")
    else:
        print("HYPOTHESIS NOT SUPPORTED: Insufficient evidence for hierarchy inversion")
        print("The traditional hierarchy appears to be maintained")

    print()

    # Additional Analysis: Detailed Hub Performance
    print("ADDITIONAL INSIGHTS: DETAILED HUB PERFORMANCE ANALYSIS")
    print("=" * 60)

    # Top and bottom performers across all hub types
    print("OVERALL TOP AND BOTTOM PERFORMERS:")
    print("-" * 40)

    top_performers = meaningful_airports.nlargest(10, 'CHANGE_PERCENT')
    bottom_performers = meaningful_airports.nsmallest(10, 'CHANGE_PERCENT')

    print("Top 10 Growth Performers:")
    for i, (_, airport) in enumerate(top_performers.iterrows(), 1):
        hub_type = airport['HUB_TYPE']
        airport_name = airport.get('AIRPORT_NAME', 'Unknown')
        print(f"{i:2d}. {airport['AIRPORT']} ({hub_type}): {airport['CHANGE_PERCENT']:+.1f}%")

    print("\nBottom 10 Performers:")
    for i, (_, airport) in enumerate(bottom_performers.iterrows(), 1):
        hub_type = airport['HUB_TYPE']
        airport_name = airport.get('AIRPORT_NAME', 'Unknown')
        print(f"{i:2d}. {airport['AIRPORT']} ({hub_type}): {airport['CHANGE_PERCENT']:+.1f}%")

    # Hub type distribution in top performers
    print(f"\nHUB TYPE DISTRIBUTION IN TOP 25% PERFORMERS:")
    print("-" * 45)

    top_quartile = meaningful_airports.nlargest(int(len(meaningful_airports) * 0.25), 'CHANGE_PERCENT')
    hub_distribution = top_quartile['HUB_TYPE'].value_counts()

    for hub_type, count in hub_distribution.items():
        total_of_type = len(meaningful_airports[meaningful_airports['HUB_TYPE'] == hub_type])
        percentage = (count / total_of_type) * 100
        print(f"{hub_type}: {count}/{total_of_type} ({percentage:.1f}%)")

    # Save results for further analysis
    try:
        correlation_df = pd.DataFrame([correlation_results]) if correlation_results else pd.DataFrame()
        correlation_df.to_csv('/content/drive/MyDrive/airline_data_analysis_v2/hyp3_outputs/hypothesis3_correlation_analysis.csv', index=False)

        performance_matrix.to_csv('/content/drive/MyDrive/airline_data_analysis_v2/hyp3_outputs/hypothesis3_performance_matrix.csv')

        concentration_df = pd.DataFrame([concentration_results])
        concentration_df.to_csv('/content/drive/MyDrive/airline_data_analysis_v2/hyp3_outputs/hypothesis3_concentration_analysis.csv', index=False)

        # Create summary
        summary_results = pd.DataFrame({
            'Criterion': ['Size-Growth Correlation', 'Hierarchy Inversion', 'Market Deconcentration'],
            'Status': [
                'MET' if correlation_results and correlation_results['pearson_corr'] < 0 and correlation_results['pearson_p'] < 0.05 else 'NOT MET',
                'MET' if hierarchy_inverted else 'NOT MET',
                'MET' if hhi_change < -10 else 'NOT MET'
            ],
            'Value': [
                f"r = {correlation_results['pearson_corr']:.3f} (p = {correlation_results['pearson_p']:.3f})" if correlation_results else "N/A",
                f"Small Hub rank: 1st" if hierarchy_inverted else "No inversion",
                f"HHI change: {hhi_change:+.1f}"
            ],
            'Interpretation': [
                'Negative correlation between size and growth',
                'Small hubs outperform larger hubs',
                'Market became less concentrated'
            ]
        })

        summary_results.to_csv('/content/drive/MyDrive/airline_data_analysis_v2/hyp3_outputs/hypothesis3_summary_results.csv', index=False)

        print("\nResults saved for subsequent analysis")
    except Exception as e:
        print(f"\nNote: Results not saved ({e})")

    print()
    print("=== ANALYSIS COMPLETE ===")
    print("Review the correlation analysis, hierarchy comparison, and concentration metrics to assess hypothesis validity.")

    return {
        'correlation_results': correlation_results,
        'performance_matrix': performance_matrix,
        'concentration_results': concentration_results,
        'hierarchy_inverted': hierarchy_inverted,
        'criteria_met': criteria_met
    }

if __name__ == "__main__":
    results = main()

=== HYPOTHESIS 3: HUB HIERARCHY INVERSION ANALYSIS ===
Testing: Traditional airport hierarchy demonstrates systematic inversion with smaller airports outperforming larger ones

Data Loading and Initial Processing...
Loaded airport analysis data: 418 airports
Loaded hub growth data: 4 hub types
Airports included in hierarchy analysis: 418
Hub types represented: ['Non-Hub Primary' 'Medium Hub' 'Small Hub' 'Major Hub']

METRIC 1: INVERSE CORRELATION ANALYSIS
Original sample size: 418
Sample size after outlier removal: 328
Outliers removed: 90 (21.5%)

CORRELATION ANALYSIS RESULTS:
----------------------------------------
Pearson correlation coefficient: 0.2690
Pearson p-value: 0.000001
Spearman correlation coefficient: 0.2450
Spearman p-value: 0.000007

CORRELATION INTERPRETATION:
Pearson correlation strength: weak
Spearman correlation strength: weak
Statistical result: Significant positive correlation - larger airports demonstrate superior growth

METRIC 2: HIERARCHICAL PERFORMANCE MATRI

# Some more statistical testing
- Paired t-tests for before/after centrality measures
- Network-level statistical tests
- Permutation tests for network metrics
- Structural equation modeling for causal pathways

In [ ]:
import os
import numpy as np
import pandas as pd

# ---- Paths ----
BASE = "/content/drive/MyDrive/airline_data_analysis_v2"
IN_T100 = os.path.join(BASE, "consolidated_t100_with_tpi_city_state.csv")
IN_AIRPORTS = os.path.join(BASE, "unique_airports_t100_hub_updated.csv")
OUT_DIR = os.path.join(BASE, "hyp3_outputs")
OUT_EDGES = os.path.join(OUT_DIR, "h3_edges_pre_post.csv")

# ---- Config ----
PRE_YEARS  = {2018, 2019}
POST_YEARS = {2023, 2024}
DOMESTIC_ONLY = True

# ---- Load ----
os.makedirs(OUT_DIR, exist_ok=True)
df = pd.read_csv(IN_T100, low_memory=False)
ap = pd.read_csv(IN_AIRPORTS)

# ---- Clean and restrict ----
df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce")
df["PASSENGERS"] = pd.to_numeric(df["PASSENGERS"], errors="coerce")
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["YEAR","PASSENGERS","ORIGIN","DEST"])

if DOMESTIC_ONLY:
    df = df[(df["ORIGIN_COUNTRY"].astype(str).str.upper().str.contains("US")) &
            (df["DEST_COUNTRY"].astype(str).str.upper().str.contains("US"))]

# ---- Build PRE / POST aggregates ----
pre = (df[df["YEAR"].isin(PRE_YEARS)]
       .groupby(["ORIGIN","DEST"], as_index=False)["PASSENGERS"].sum()
       .rename(columns={"PASSENGERS":"W_PRE"}))

post = (df[df["YEAR"].isin(POST_YEARS)]
        .groupby(["ORIGIN","DEST"], as_index=False)["PASSENGERS"].sum()
        .rename(columns={"PASSENGERS":"W_POST"}))

# ---- Merge into single edge list ----
edges = pd.merge(pre, post, on=["ORIGIN","DEST"], how="outer")
edges["W_PRE"]  = edges["W_PRE"].fillna(0).astype(float)
edges["W_POST"] = edges["W_POST"].fillna(0).astype(float)
edges = edges[edges["ORIGIN"] != edges["DEST"]].copy()

# ---- Growth calculations ----
edges["GROWTH_ABS"] = edges["W_POST"] - edges["W_PRE"]
with np.errstate(divide="ignore", invalid="ignore"):
    edges["GROWTH_PCT"] = np.where(
        edges["W_PRE"] > 0,
        100.0 * edges["GROWTH_ABS"] / edges["W_PRE"],
        np.nan
    )

# ---- Enrich with hub classifications ----
ap_hubs = ap[["AIRPORT_NAME","HUB_TYPE"]].rename(columns={"AIRPORT_NAME":"ORIGIN","HUB_TYPE":"ORIGIN_HUB_TYPE"})
edges = edges.merge(ap_hubs, on="ORIGIN", how="left")

ap_hubs = ap[["AIRPORT_NAME","HUB_TYPE"]].rename(columns={"AIRPORT_NAME":"DEST","HUB_TYPE":"DEST_HUB_TYPE"})
edges = edges.merge(ap_hubs, on="DEST", how="left")

# ---- Save ----
edges = edges.sort_values(["ORIGIN","DEST"]).reset_index(drop=True)
edges.to_csv(OUT_EDGES, index=False)
print(f"Saved: {OUT_EDGES}  (rows={len(edges)})")


Saved: /content/drive/MyDrive/airline_data_analysis_v2/hyp3_outputs/h3_edges_pre_post.csv  (rows=40010)


In [ ]:
# -*- coding: utf-8 -*-
"""
H3 — Network Analysis & Report Generator (GPU/RAM aware + renumbered cuGraph)
Outputs:
  /content/drive/MyDrive/airline_data_analysis_v2/hyp3_outputs/hyp3_stats_report.md
  /content/drive/MyDrive/airline_data_analysis_v2/hyp3_outputs/hyp3_summary.json
"""

import os, json, time, math, sys, subprocess
import numpy as np
import pandas as pd
import networkx as nx
from scipy import stats
from typing import Callable, Dict, List

# ---------------- CONFIG ----------------
ROOT_DIR    = "/content/drive/MyDrive/airline_data_analysis_v2"
INPUT_EDGES = os.path.join(ROOT_DIR, "hyp3_outputs", "h3_edges_pre_post.csv")
OUTPUT_DIR  = os.path.join(ROOT_DIR, "hyp3_outputs")

# Default sizes (auto-tuned higher if GPU & high-RAM)
N_BOOT      = 500
N_PERM      = 1000
K_BETWEENNESS = 64
ALPHA       = 0.05
PRINT_EVERY = 100
# ----------------------------------------

# ---- Environment checks (GPU & RAM) ----
print("=== Environment ===")
try:
    import psutil
    ram_gb = psutil.virtual_memory().total / 1e9
    print(f"RAM: {ram_gb:.1f} GB")
except Exception:
    ram_gb = None
    print("psutil not available; skipping RAM check.")

try:
    out = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
    if out.returncode == 0 and "NVIDIA-SMI" in out.stdout:
        print("GPU detected:\n", out.stdout.splitlines()[0])
        HAS_GPU = True
    else:
        print("No GPU detected.")
        HAS_GPU = False
except Exception:
    print("No GPU detected.")
    HAS_GPU = False

# Auto-tune sizes if strong machine
if HAS_GPU and (ram_gb is None or ram_gb >= 20):
    N_BOOT = max(N_BOOT, 1000)
    N_PERM = max(N_PERM, 2000)
    K_BETWEENNESS = max(K_BETWEENNESS, 128)
    print(f"Auto-tuned for GPU/high-RAM: N_BOOT={N_BOOT}, N_PERM={N_PERM}, K_BETWEENNESS={K_BETWEENNESS}")
else:
    print(f"Using conservative sizes: N_BOOT={N_BOOT}, N_PERM={N_PERM}, K_BETWEENNESS={K_BETWEENNESS}")

# Try to enable RAPIDS if GPU present
HAS_RAPIDS = False
if HAS_GPU:
    try:
        import cudf, cugraph
        HAS_RAPIDS = True
        print("RAPIDS (cuDF/cuGraph) already available.")
    except Exception:
        print("Installing RAPIDS (cuDF/cuGraph) for CUDA 12 ...")
        pkgs = ["cudf-cu12", "cugraph-cu12", "cupy-cuda12x"]
        subprocess.run([sys.executable, "-m", "pip", "install", "-q"] + pkgs, check=False)
        try:
            import cudf, cugraph
            HAS_RAPIDS = True
            print("RAPIDS installed and imported.")
        except Exception as e:
            print("RAPIDS install failed; falling back to CPU.", e)

# Optional: community modularity (python-louvain). If missing, modularity will be NaN.
try:
    import community  # python-louvain
    _HAS_COMMUNITY = True
except Exception:
    _HAS_COMMUNITY = False

def ensure_dir(p):
    if not os.path.exists(p): os.makedirs(p, exist_ok=True)

# ---------------- GRAPH HELPERS (CPU) ----------------
def build_weighted_digraph_pandas(edges: pd.DataFrame,
                                  wcol: str,
                                  s_col: str = "ORIGIN",
                                  t_col: str = "DEST",
                                  remove_self_loops: bool = True) -> nx.DiGraph:
    G = nx.DiGraph()
    add = G.add_edge
    for u, v, w in edges[[s_col, t_col, wcol]].itertuples(index=False, name=None):
        if remove_self_loops and u == v: continue
        if G.has_edge(u, v):
            G[u][v]["weight"] += float(w)
        else:
            add(u, v, weight=float(w))
    return G

# ---------- cuGraph builders (renumber=True) ----------
def build_cugraph_digraph(edges: "cudf.DataFrame",
                          wcol: str,
                          s_col: str = "ORIGIN",
                          t_col: str = "DEST",
                          remove_self_loops: bool = True):
    """
    Build a directed cuGraph.Graph with renumber=True (required when vertex cols are non-integer).
    Returns (G, renumber_map), where renumber_map maps internal vertex ids -> original ids.
    """
    import cudf, cugraph
    df = edges[[s_col, t_col, wcol]].copy()
    if remove_self_loops:
        df = df[df[s_col] != df[t_col]]

    # Ensure weight column name 'weight'
    df = df.rename(columns={wcol: "weight"})

    G = cugraph.Graph(directed=True)
    # renumber=True lets cuGraph assign internal int ids even if ORIGIN/DEST are strings
    G.from_cudf_edgelist(df, source=s_col, destination=t_col,
                         edge_attr="weight", renumber=True)
    # Get renumber map (internal -> original ids). API varies slightly across versions, so be defensive.
    renum = None
    try:
        renum = G.renumber_map  # Series or DataFrame depending on version
    except Exception:
        pass
    return G, renum

def _unrenumber_df(df, col, renum):
    """Try to unrenumber a DataFrame 'df' in-place given a renumber map; safe no-op if unavailable."""
    try:
        import cugraph
        from cugraph.utilities import unrenumber
        return unrenumber(df, col, renum)
    except Exception:
        return df  # fallback

def get_edgelist_df(G_cu):
    """
    Fetch a cuDF edgelist from a cuGraph Graph across RAPIDS versions and
    normalize column names to: 'src', 'dst', 'weight'.

    Robust to odd orders/names like ['weight','ORIGIN','DEST'] or
    absence of explicit 'weight' (creates unit weights).
    """
    import cudf

    # 1) Try current accessor
    df = None
    try:
        df = G_cu.view_edge_list()
    except Exception:
        pass

    # 2) Try legacy accessor
    if df is None:
        try:
            df = G_cu.edgelist.edgelist_df
        except Exception as e:
            raise RuntimeError("Cannot retrieve cuGraph edgelist") from e

    cols = list(df.columns)

    # Candidates to recognize by name
    src_candidates = {
        "src","source","sources","from","source_vertex","source_id",
        "src_0","ORIGIN","Origin","origin"
    }
    dst_candidates = {
        "dst","destination","destinations","to","dest_vertex","dest_id",
        "dst_0","DEST","Dest","dest"
    }
    w_candidates = {"weight","weights","edge_weight","w","W","value","WEIGHT"}

    # Identify weight by name first
    w_col = next((c for c in cols if c in w_candidates), None)

    # If no explicit weight, try to infer:
    if w_col is None:
        # Heuristic: if exactly 3 cols and one looks numeric, pick the numeric one.
        if len(cols) == 3:
            numeric_like = []
            for c in cols:
                try:
                    # cuDF dtype kind 'f'/'i'/'u' are numeric; fall back to try astype
                    if str(df[c].dtype).startswith(("float","int","uint")):
                        numeric_like.append(c)
                except Exception:
                    pass
            if len(numeric_like) == 1:
                w_col = numeric_like[0]

    # If still none, create a unit weight
    if w_col is None:
        df = df.assign(weight=1.0)
        w_col = "weight"

    # Now find src/dst by name
    src_col = next((c for c in cols if c in src_candidates), None)
    dst_col = next((c for c in cols if c in dst_candidates), None)

    # If either is missing, infer by "the two non-weight columns"
    if src_col is None or dst_col is None:
        non_weight = [c for c in cols if c != w_col]
        if len(non_weight) < 2:
            raise RuntimeError(f"Cannot infer src/dst; edgelist columns={cols}")
        # Keep original order among non-weight columns; treat first as src, second as dst
        src_col, dst_col = non_weight[0], non_weight[1]

    # Normalize names
    df_norm = df.rename(columns={src_col: "src", dst_col: "dst", w_col: "weight"})
    # Keep only needed cols in canonical order
    return df_norm[["src", "dst", "weight"]]



def compute_centralities_gpu(G_cu,
                             renum_map=None,
                             pagerank_alpha: float = 0.85,
                             k_sources: int = 128) -> Dict[str, Dict]:
    """
    GPU centralities using cuGraph + cuDF with version-robust edgelist handling.
    - Weighted degree/in/out via cuDF groupbys on normalized edgelist (src,dst,weight)
    - PageRank / Betweenness / Eigenvector via cuGraph
    - Unrenumber outputs back to original IDs when renum_map is provided
    """
    import cudf, cugraph

    out = {}

    # --- Normalize the edgelist view (handles all RAPIDS variants) ---
    view = get_edgelist_df(G_cu)  # columns: src, dst, weight
    print(f"GPU centralities: using edgelist columns {list(view.columns)}")

    # --- Weighted degrees via cuDF groupby ---
    out_deg = view.groupby("src")["weight"].sum().reset_index().rename(columns={"weight": "out_degree", "src": "vertex"})
    in_deg  = view.groupby("dst")["weight"].sum().reset_index().rename(columns={"weight": "in_degree",  "dst": "vertex"})
    deg_df  = out_deg.merge(in_deg, on="vertex", how="outer").fillna(0.0)
    deg_df["degree"] = deg_df["in_degree"] + deg_df["out_degree"]

    # Unrenumber util (safe no-op if not available)
    def _unrenumber_df(df, col, renum):
        try:
            from cugraph.utilities import unrenumber
            return unrenumber(df, col, renum)
        except Exception:
            return df

    deg_df  = _unrenumber_df(deg_df,  "vertex", renum_map)
    out_deg = _unrenumber_df(out_deg, "vertex", renum_map)
    in_deg  = _unrenumber_df(in_deg,  "vertex", renum_map)

    out["degree"]     = dict(zip(deg_df["vertex"].to_pandas(),  deg_df["degree"].to_pandas()))
    out["in_degree"]  = dict(zip(in_deg["vertex"].to_pandas(),  in_deg["in_degree"].to_pandas()))
    out["out_degree"] = dict(zip(out_deg["vertex"].to_pandas(), out_deg["out_degree"].to_pandas()))

    # --- PageRank ---
    print("GPU centralities: pagerank ...")
    pr_df = cugraph.pagerank(G_cu, alpha=pagerank_alpha)
    pr_df = _unrenumber_df(pr_df, "vertex", renum_map)
    out["pagerank"] = dict(zip(pr_df["vertex"].to_pandas(), pr_df["pagerank"].to_pandas()))

    # --- Betweenness (approx with k sources) ---
    print(f"GPU centralities: betweenness approx (k={k_sources}) ...")
    try:
        k = min(k_sources, int(G_cu.number_of_vertices()))
        bc_df = cugraph.betweenness_centrality(G_cu, k=k, normalized=True)
        bc_df = _unrenumber_df(bc_df, "vertex", renum_map)
        bc_col = "betweenness_centrality" if "betweenness_centrality" in bc_df.columns else (
                 "betweenness" if "betweenness" in bc_df.columns else bc_df.columns[-1])
        out["betweenness"] = dict(zip(bc_df["vertex"].to_pandas(), bc_df[bc_col].to_pandas()))
    except Exception:
        out["betweenness"] = {}

    # --- Eigenvector (needs undirected proxy) ---
    print("GPU centralities: eigenvector (undirected proxy) ...")
    try:
        Gu = cugraph.Graph(directed=False)
        Gu.from_cudf_edgelist(view, source="src", destination="dst", edge_attr="weight", renumber=False)
        ev_df = cugraph.eigenvector_centrality(Gu)
        ev_df = _unrenumber_df(ev_df, "vertex", renum_map)
        ev_col = "eigenvector_centrality" if "eigenvector_centrality" in ev_df.columns else ev_df.columns[-1]
        out["eigenvector"] = dict(zip(ev_df["vertex"].to_pandas(), ev_df[ev_col].to_pandas()))
    except Exception:
        out["eigenvector"] = {}

    return out



# --------------- CENTRALITIES (CPU) ---------------
def compute_centralities_cpu(G: nx.DiGraph,
                             use_approx_betw=True,
                             k_sources=K_BETWEENNESS,
                             pagerank_alpha=0.85) -> Dict[str, Dict]:
    w = "weight"
    print("CPU centralities: degree/in/out ...")
    deg   = dict(G.degree(weight=w))
    indeg = dict(G.in_degree(weight=w))
    outd  = dict(G.out_degree(weight=w))
    if use_approx_betw:
        print(f"CPU centralities: betweenness approx (k={k_sources}) ...")
        nodes = list(G.nodes)
        if len(nodes) == 0:
            src = []
        else:
            rng = np.random.default_rng(123)
            src = list(rng.choice(nodes, size=min(k_sources, len(nodes)), replace=False))
        try:
            btw = nx.betweenness_centrality_subset(G, sources=src, targets=list(G.nodes), normalized=True, weight=w)
        except Exception:
            btw = {n: np.nan for n in G.nodes}
    else:
        print("CPU centralities: betweenness exact (slow) ...")
        btw = nx.betweenness_centrality(G, weight=w, normalized=True)
    print("CPU centralities: eigenvector (undirected proxy) ...")
    try:
        ev = nx.eigenvector_centrality_numpy(G.to_undirected(), weight=w)
    except Exception:
        ev = {n: np.nan for n in G.nodes}
    print("CPU centralities: pagerank ...")
    try:
        pr = nx.pagerank(G, alpha=pagerank_alpha, weight=w)
    except Exception:
        pr = {n: np.nan for n in G.nodes}
    return {"degree":deg, "in_degree":indeg, "out_degree":outd,
            "betweenness":btw, "eigenvector":ev, "pagerank":pr}

def ensure_common_nodes(G_pre: nx.DiGraph, G_post: nx.DiGraph) -> List:
    nodes = sorted(set(G_pre.nodes()) | set(G_post.nodes()))
    for n in nodes:
        if n not in G_pre:  G_pre.add_node(n)
        if n not in G_post: G_post.add_node(n)
    return nodes

def paired_centrality_tests(G_pre, G_post, use_gpu=False, metrics=None) -> pd.DataFrame:
    print("[1/4] Paired centrality tests: start")
    t0 = time.time()
    if metrics is None:
        metrics = ["degree","in_degree","out_degree","betweenness","eigenvector","pagerank"]

    if use_gpu and HAS_RAPIDS:
        import cudf
        print("  - building cuGraph views (with renumber=True) ...")
        # Convert NetworkX graphs to cuDF
        def nx_to_cudf(Gnx):
            src, dst, w = [], [], []
            for u, v, d in Gnx.edges(data=True):
                src.append(u); dst.append(v); w.append(d.get("weight", 1.0))
            return cudf.DataFrame({"ORIGIN":src, "DEST":dst, "W":w})
        df_pre  = nx_to_cudf(G_pre)
        df_post = nx_to_cudf(G_post)
        G0, ren0 = build_cugraph_digraph(df_pre.rename(columns={"W":"W_PRE"}),  "W_PRE")
        G1, ren1 = build_cugraph_digraph(df_post.rename(columns={"W":"W_POST"}), "W_POST")
        Cpre  = compute_centralities_gpu(G0, renum_map=ren0, k_sources=K_BETWEENNESS)
        Cpost = compute_centralities_gpu(G1, renum_map=ren1, k_sources=K_BETWEENNESS)
        nodes = set()
        for key in ("degree","pagerank","in_degree","out_degree","betweenness","eigenvector"):
            nodes |= set(Cpre.get(key,{}).keys()) | set(Cpost.get(key,{}).keys())
        nodes = sorted(nodes)
    else:
        nodes = ensure_common_nodes(G_pre, G_post)
        print(f"  - nodes in union: {len(nodes)}")
        print("  - computing centralities (Pre)")
        Cpre  = compute_centralities_cpu(G_pre, use_approx_betw=True, k_sources=K_BETWEENNESS)
        print("  - computing centralities (Post)")
        Cpost = compute_centralities_cpu(G_post, use_approx_betw=True, k_sources=K_BETWEENNESS)

    rows=[]
    for m in metrics:
        print(f"  - testing metric: {m}")
        x = np.array([Cpre.get(m,{}).get(n, np.nan)  for n in nodes], dtype=float)
        y = np.array([Cpost.get(m,{}).get(n, np.nan) for n in nodes], dtype=float)
        mask = np.isfinite(x) & np.isfinite(y)
        x, y = x[mask], y[mask]
        if len(x) < 3:
            rows.append({"metric":m,"n":len(x),"t":np.nan,"p_t":np.nan,"dz":np.nan,
                         "wilcoxon_stat":np.nan,"p_w":np.nan})
            continue
        t_stat, p_t = stats.ttest_rel(x, y, nan_policy="omit")
        d = y - x
        dz = np.mean(d) / (np.std(d, ddof=1) + 1e-12)  # Cohen's dz
        try:
            w_stat, p_w = stats.wilcoxon(x, y, zero_method="wilcox", correction=False)
        except Exception:
            w_stat, p_w = np.nan, np.nan
        rows.append({"metric":m,"n":len(x),"t":t_stat,"p_t":p_t,"dz":dz,
                     "wilcoxon_stat":w_stat,"p_w":p_w})
    print(f"[1/4] Paired centrality tests: done in {time.time()-t0:.1f}s")
    return pd.DataFrame(rows)

# --------------- NETWORK METRICS ---------------
def network_level_metrics(G: nx.DiGraph) -> Dict[str, float]:
    res={}
    res["n_nodes"] = G.number_of_nodes()
    res["n_edges"] = G.number_of_edges()
    res["density"] = nx.density(G)
    if G.number_of_nodes()==0:
        return res
    comp_nodes = max(nx.weakly_connected_components(G), key=len) if G.is_directed() else max(nx.connected_components(G), key=len)
    H = G.subgraph(comp_nodes).copy()
    try:
        res["transitivity"]   = nx.transitivity(H.to_undirected())
        res["avg_clustering"] = nx.average_clustering(H.to_undirected(), weight="weight")
    except Exception:
        res["transitivity"] = np.nan; res["avg_clustering"] = np.nan
    try:
        W = H.copy()
        for u,v,data in W.edges(data=True):
            w = data.get("weight",1.0)
            data["cost"] = 1.0/w if w>0 else 1e9
        res["aspl_cost"] = nx.average_shortest_path_length(W, weight="cost") if nx.is_weakly_connected(W) else np.nan
    except Exception:
        res["aspl_cost"] = np.nan
    try:
        res["assortativity_deg"] = nx.degree_assortativity_coefficient(H.to_undirected())
    except Exception:
        res["assortativity_deg"] = np.nan
    if _HAS_COMMUNITY:
        try:
            part = community.best_partition(H.to_undirected(), weight="weight")
            res["modularity"] = community.modularity(part, H.to_undirected(), weight="weight")
        except Exception:
            res["modularity"] = np.nan
    else:
        res["modularity"] = np.nan
    return res

def bootstrap_metric_diff(edges: pd.DataFrame,
                          metric_fn: Callable[[nx.DiGraph], float],
                          n_boot: int = N_BOOT,
                          s_col: str = "ORIGIN",
                          t_col: str = "DEST",
                          seed: int = 42) -> Dict[str, float]:
    print(f"  - bootstrap CI ({metric_fn.__name__}): n_boot={n_boot}")
    rng = np.random.default_rng(seed)
    G0 = build_weighted_digraph_pandas(edges, "W_PRE",  s_col, t_col)
    G1 = build_weighted_digraph_pandas(edges, "W_POST", s_col, t_col)
    diff_obs = metric_fn(G1) - metric_fn(G0)
    diffs=[]; idx=np.arange(len(edges))
    t0 = time.time()
    for b in range(n_boot):
        if (b+1) % PRINT_EVERY == 0:
            print(f"    * bootstrap {b+1}/{n_boot} (elapsed {time.time()-t0:.1f}s)")
        samp = edges.iloc[rng.choice(idx, size=len(idx), replace=True)]
        Gp = build_weighted_digraph_pandas(samp, "W_PRE",  s_col, t_col)
        Gq = build_weighted_digraph_pandas(samp, "W_POST", s_col, t_col)
        diffs.append(metric_fn(Gq) - metric_fn(Gp))
    diffs = np.array(diffs, dtype=float)
    lo,hi = np.quantile(diffs,[0.025,0.975])
    return {"diff_obs": float(diff_obs), "ci95_lo": float(lo), "ci95_hi": float(hi)}

def perm_test_network_metric(edges: pd.DataFrame,
                             metric_fn: Callable[[nx.DiGraph], float],
                             n_perm: int = N_PERM,
                             s_col: str = "ORIGIN",
                             t_col: str = "DEST",
                             seed: int = 123) -> Dict[str, float]:
    print(f"  - permutation test ({metric_fn.__name__}): n_perm={n_perm}")
    rng = np.random.default_rng(seed)
    G0 = build_weighted_digraph_pandas(edges, "W_PRE",  s_col, t_col)
    G1 = build_weighted_digraph_pandas(edges, "W_POST", s_col, t_col)
    diff_obs = metric_fn(G1) - metric_fn(G0)
    diffs = np.empty(n_perm, dtype=float)
    base = edges[["ORIGIN","DEST","W_PRE","W_POST"]].to_numpy(copy=True)
    t0 = time.time()
    for i in range(n_perm):
        if (i+1) % PRINT_EVERY == 0:
            print(f"    * perm {i+1}/{n_perm} (elapsed {time.time()-t0:.1f}s)")
        arr = base.copy()
        swaps = rng.random(len(arr)) < 0.5
        arr[swaps, [2,3]] = arr[swaps, [3,2]]  # swap per edge
        eperm = pd.DataFrame(arr, columns=["ORIGIN","DEST","W_PRE","W_POST"])
        Gp = build_weighted_digraph_pandas(eperm, "W_PRE",  s_col, t_col)
        Gq = build_weighted_digraph_pandas(eperm, "W_POST", s_col, t_col)
        diffs[i] = metric_fn(Gq) - metric_fn(Gp)
    p_two = (1.0 + np.sum(np.abs(diffs) >= np.abs(diff_obs))) / (n_perm + 1.0)
    return {"diff_obs": float(diff_obs), "p_two_sided": float(p_two)}

# Metric wrappers
def metric_avg_clustering(G): return network_level_metrics(G).get("avg_clustering", np.nan)
def metric_aspl_cost(G):     return network_level_metrics(G).get("aspl_cost", np.nan)
def metric_modularity(G):    return network_level_metrics(G).get("modularity", np.nan)

# --------------- SEM ---------------
def run_sem_node_level(G_pre: nx.DiGraph, G_post: nx.DiGraph) -> Dict:
    print("[4/4] SEM: start")
    t0 = time.time()
    try:
        import semopy
        print("  - semopy found")
    except Exception:
        print("  - installing semopy ...")
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "semopy"], check=False)
        import semopy
        print("  - semopy installed")

    nodes = ensure_common_nodes(G_pre, G_post)
    print(f"  - SEM nodes: {len(nodes)} (features)")

    def node_strength(G):
        s_in  = dict(G.in_degree(weight="weight"))
        s_out = dict(G.out_degree(weight="weight"))
        return {n: s_in.get(n,0.0) + s_out.get(n,0.0) for n in nodes}

    try:
        ev_pre  = nx.eigenvector_centrality_numpy(G_pre.to_undirected(),  weight="weight")
        ev_post = nx.eigenvector_centrality_numpy(G_post.to_undirected(), weight="weight")
    except Exception:
        ev_pre  = {n: np.nan for n in nodes}
        ev_post = {n: np.nan for n in nodes}

    s_pre  = node_strength(G_pre)
    s_post = node_strength(G_post)

    df = pd.DataFrame({
        "node": nodes,
        "centrality_pre":  [ev_pre.get(n, np.nan)  for n in nodes],
        "centrality_post": [ev_post.get(n, np.nan) for n in nodes],
        "strength_pre":    [s_pre.get(n, 0.0)      for n in nodes],
        "strength_post":   [s_post.get(n, 0.0)     for n in nodes],
    })
    df["centrality_change"] = df["centrality_post"] - df["centrality_pre"]
    with np.errstate(divide='ignore', invalid='ignore'):
        df["growth"] = 100.0 * (df["strength_post"] - df["strength_pre"]) / np.where(df["strength_pre"]>0, df["strength_pre"], np.nan)

    X = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["centrality_pre","centrality_post","centrality_change","strength_pre","growth"]).copy()
    print(f"  - SEM rows used: {len(X)}")

    # Standardize numeric cols
    for c in X.select_dtypes(include=[np.number]).columns:
        s = X[c].std(ddof=0)
        if s>0: X[c] = (X[c] - X[c].mean())/s

    model_desc = """
    centrality_post ~ centrality_pre + strength_pre
    growth          ~ centrality_change + strength_pre
    """
    model = semopy.Model(model_desc)
    opt   = semopy.Optimizer(model)
    opt.optimize(X)
    est = model.inspect(std_est=True)
    try:
        fit = semopy.calc_stats(model, X)
        fit_keep = {k: fit.get(k) for k in ("AIC","BIC","CFI","TLI","RMSEA","SRMR","GFI","AGFI")}
    except Exception:
        fit_keep = {}
    print(f"[4/4] SEM: done in {time.time()-t0:.1f}s")
    return {"data_used_n": int(len(X)), "estimates": est.to_dict(orient="records"), "fit": fit_keep}

# --------------- REPORT UTILS ---------------
def fmt(x, nd=3):
    if x is None or (isinstance(x,float) and (np.isnan(x) or not np.isfinite(x))): return "NA"
    return f"{x:.{nd}f}"

def fmt_p(p):
    if p is None or (isinstance(p,float) and (np.isnan(p) or not np.isfinite(p))): return "p=NA"
    if p < 1e-4: return "p<1e-4"
    return f"p={p:.4f}"

def section(title): return f"\n## {title}\n"

# ---------------------- MAIN ----------------------
def main():
    ensure_dir(OUTPUT_DIR)
    assert os.path.exists(INPUT_EDGES), f"Edges file not found: {INPUT_EDGES}"
    print(f"Reading edges: {INPUT_EDGES}")
    edges = pd.read_csv(INPUT_EDGES)
    print(f"  - rows: {len(edges)} | columns: {list(edges.columns)}")

    # Build CPU graphs
    print("Building CPU graphs ...")
    t0 = time.time()
    G_pre  = build_weighted_digraph_pandas(edges, "W_PRE")
    G_post = build_weighted_digraph_pandas(edges, "W_POST")
    print(f"  - G_pre: nodes={G_pre.number_of_nodes()}, edges={G_pre.number_of_edges()}")
    print(f"  - G_post: nodes={G_post.number_of_nodes()}, edges={G_post.number_of_edges()}")
    print(f"Graphs built in {time.time()-t0:.1f}s")

    # 1) Paired centrality tests (GPU if RAPIDS available, using renumber/unrenumber)
    ct_df = paired_centrality_tests(G_pre, G_post, use_gpu=HAS_RAPIDS)

    # 2) Network-level metrics + bootstrap CIs
    print("[2/4] Network-level metrics (baseline) ...")
    t2 = time.time()
    metrics_then = network_level_metrics(G_pre)
    metrics_now  = network_level_metrics(G_post)
    print(f"  - baseline computed in {time.time()-t2:.1f}s")

    print("[2/4] Bootstrap CIs ...")
    t2 = time.time()
    boots = {
        "avg_clustering": bootstrap_metric_diff(edges, metric_avg_clustering, n_boot=N_BOOT),
        "aspl_cost":      bootstrap_metric_diff(edges, metric_aspl_cost,      n_boot=N_BOOT),
        "modularity":     bootstrap_metric_diff(edges, metric_modularity,     n_boot=N_BOOT) if _HAS_COMMUNITY else {"diff_obs": np.nan, "ci95_lo": np.nan, "ci95_hi": np.nan}
    }
    print(f"[2/4] Bootstrap done in {time.time()-t2:.1f}s")

    # 3) Permutation tests
    print("[3/4] Permutation tests ...")
    t3 = time.time()
    perms = {
        "avg_clustering": perm_test_network_metric(edges, metric_avg_clustering, n_perm=N_PERM),
        "aspl_cost":      perm_test_network_metric(edges, metric_aspl_cost,      n_perm=N_PERM),
        "modularity":     perm_test_network_metric(edges, metric_modularity,     n_perm=N_PERM) if _HAS_COMMUNITY else {"diff_obs": np.nan, "p_two_sided": np.nan}
    }
    print(f"[3/4] Permutation done in {time.time()-t3:.1f}s")

    # Plain-English summaries
    try:
        top_change = ct_df.sort_values("dz", key=lambda s: s.abs(), ascending=False).iloc[0]
        plain_ct = f"The biggest shift was in **{top_change['metric']}** (effect size dz≈{fmt(top_change['dz'])})."
    except Exception:
        plain_ct = "Centrality changes were modest overall."

    plain_boot = []
    for name, res in boots.items():
        if all(np.isfinite([res.get("diff_obs",np.nan), res.get("ci95_lo",np.nan), res.get("ci95_hi",np.nan)])):
            crosses = res["ci95_lo"] <= 0.0 <= res["ci95_hi"]
            plain_boot.append(f"- **{name}** changed by {fmt(res['diff_obs'])} (95% CI [{fmt(res['ci95_lo'])}, {fmt(res['ci95_hi'])}]) — {'uncertain (CI crosses 0)' if crosses else 'likely real (CI excludes 0)'}")
        else:
            plain_boot.append(f"- **{name}**: CI not available.")
    plain_boot = "\n".join(plain_boot)

    plain_perm = []
    for name, res in perms.items():
        ptxt = fmt_p(res.get("p_two_sided", np.nan))
        plain_perm.append(f"- **{name}**: observed change={fmt(res.get('diff_obs',np.nan))}, {ptxt} → {'unlikely by chance' if (isinstance(res.get('p_two_sided',np.nan),float) and res['p_two_sided']<ALPHA) else 'could be chance variation'}")
    plain_perm = "\n".join(plain_perm)

    # 4) SEM
    sem_out = run_sem_node_level(G_pre, G_post)
    sem_plain = f"SEM used {sem_out['data_used_n']} nodes. Paths estimate how prior connectivity relates to later connectivity and growth; significant positive coefficients indicate airports with higher starting centrality (or strength) tended to remain central or grow more."

    # Save JSON summary
    summary = {
        "inputs": {"edges_path": INPUT_EDGES},
        "centrality_tests": ct_df.to_dict(orient="records"),
        "network_metrics_pre": metrics_then,
        "network_metrics_post": metrics_now,
        "bootstrap_diffs": boots,
        "perm_tests": perms,
        "sem": sem_out,
        "notes": "GPU centralities use renumber=True and unrenumbered back to original node ids."
    }
    out_json = os.path.join(OUTPUT_DIR, "hyp3_summary.json")
    with open(out_json, "w") as f:
        json.dump(summary, f, indent=2)
    print(f"Wrote JSON: {out_json}")

    # Build Markdown report
    md = []
    md.append("# Hypothesis 3 — Network Inversion Analyses\n")
    md.append(f"- **Edges source:** `{INPUT_EDGES}`")
    md.append(f"- **GPU:** {'ON (RAPIDS)' if HAS_RAPIDS else ('ON (no RAPIDS)' if HAS_GPU else 'OFF')}  |  **RAM:** {('%.1f GB' % ram_gb) if ram_gb else 'NA'}\n")

    md.append(section("Paired t-tests on Node Centralities (Pre vs Post)"))
    md.append(pd.DataFrame(summary["centrality_tests"]).to_markdown(index=False))
    md.append("**Technical interpretation:** For each centrality metric, paired t-tests (and Wilcoxon) test whether nodes changed from Pre to Post; Cohen’s dz summarizes effect size.\n")
    md.append(f"**In plain English:** {plain_ct}\n")

    md.append(section("Network-Level Metrics (Pre vs Post Baseline)"))
    pre_tbl  = pd.DataFrame(summary["network_metrics_pre"],  index=["Pre"]).T.rename(columns={"Pre":"value"})
    post_tbl = pd.DataFrame(summary["network_metrics_post"], index=["Post"]).T.rename(columns={"Post":"value"})
    md.append("**Pre (baseline):**");  md.append(pre_tbl.to_markdown())
    md.append("**Post:**");            md.append(post_tbl.to_markdown())
    md.append("**Technical interpretation:** Density, clustering, path length, assortativity, and modularity describe global structure.\n")
    md.append("**In plain English:** Compare Pre vs Post to see if the network became more connected (higher clustering/density) or more streamlined (lower path cost).\n")

    md.append(section("Bootstrap CIs for Metric Differences (Post − Pre)"))
    md.append(pd.DataFrame(summary["bootstrap_diffs"]).T.to_markdown())
    md.append("**Technical interpretation:** Nonparametric bootstrap resamples OD pairs; CI excluding 0 implies a likely real change.\n")
    md.append(f"**In plain English:**\n{plain_boot}\n")

    md.append(section("Permutation Tests (Paired Label-Swap)"))
    md.append(pd.DataFrame(summary["perm_tests"]).T.to_markdown())
    md.append("**Technical interpretation:** Swapping (W_PRE,W_POST) per OD forms a null distribution; two-sided p-value tests if the observed difference is unusual.\n")
    md.append(f"**In plain English:**\n{plain_perm}\n")

    md.append(section("Structural Equation Modeling (SEM) — Node-Level"))
    est_df = pd.DataFrame(sem_out["estimates"])
    if not est_df.empty:
        md.append("**Parameter estimates (standardized):**")
        md.append(est_df.to_markdown(index=False))
    else:
        md.append("_No SEM estimates available (insufficient data after cleaning)._")
    fit_df = pd.DataFrame([sem_out.get("fit", {})]).T
    if not fit_df.empty:
        md.append("**Fit indices:**"); md.append(fit_df.to_markdown(header=False))
    md.append("**Technical interpretation:** Paths from centrality_pre/strength_pre to centrality_post test persistence; paths to growth test whether centrality changes relate to traffic growth.\n")
    md.append(f"**In plain English:** {sem_plain}\n")

    out_md = os.path.join(OUTPUT_DIR, "hyp3_stats_report.md")
    with open(out_md, "w") as f:
        f.write("\n".join(md))
    print(f"Wrote MD: {out_md}")

if __name__ == "__main__":
    t_all = time.time()
    main()
    print(f"ALL DONE in {time.time()-t_all:.1f}s")


=== Environment ===
RAM: 56.9 GB
GPU detected:
 Tue Sep  2 00:03:13 2025       
Auto-tuned for GPU/high-RAM: N_BOOT=1000, N_PERM=2000, K_BETWEENNESS=128
RAPIDS (cuDF/cuGraph) already available.
Reading edges: /content/drive/MyDrive/airline_data_analysis_v2/hyp3_outputs/h3_edges_pre_post.csv
  - rows: 40010 | columns: ['ORIGIN', 'DEST', 'W_PRE', 'W_POST', 'GROWTH_ABS', 'GROWTH_PCT', 'ORIGIN_HUB_TYPE', 'DEST_HUB_TYPE']
Building CPU graphs ...
  - G_pre: nodes=1632, edges=39957
  - G_post: nodes=1632, edges=39957
Graphs built in 0.4s
[1/4] Paired centrality tests: start
  - building cuGraph views (with renumber=True) ...
GPU centralities: using edgelist columns ['src', 'dst', 'weight']
GPU centralities: pagerank ...


/usr/local/lib/python3.12/dist-packages/cugraph/link_analysis/pagerank.py:232: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


GPU centralities: betweenness approx (k=128) ...
GPU centralities: eigenvector (undirected proxy) ...
GPU centralities: using edgelist columns ['src', 'dst', 'weight']
GPU centralities: pagerank ...
GPU centralities: betweenness approx (k=128) ...


/usr/local/lib/python3.12/dist-packages/cugraph/link_analysis/pagerank.py:232: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


GPU centralities: eigenvector (undirected proxy) ...
  - testing metric: degree
  - testing metric: in_degree
  - testing metric: out_degree
  - testing metric: betweenness
  - testing metric: eigenvector
  - testing metric: pagerank
[1/4] Paired centrality tests: done in 1.2s
[2/4] Network-level metrics (baseline) ...
  - baseline computed in 9.0s
[2/4] Bootstrap CIs ...
  - bootstrap CI (metric_avg_clustering): n_boot=1000
